In [1]:
import pandas as pd
import requests
import pickle
import time
from bs4 import BeautifulSoup as bs
from tqdm import tqdm

In [2]:
def getHtmlToSoup(url):
    webdriverhtml = requests.get(url)
    return bs(webdriverhtml.text,'html.parser')

In [3]:
#찾을 urls
f = open('../data/urls/react.pkl','rb')
urls = pickle.load(f)
print(*urls[:5], sep='\n')
print('...')

https://reactjs.org
https://reactjs.org/docs/getting-started.html
https://reactjs.org/tutorial/tutorial.html
https://reactjs.org/blog/
https://reactjs.org/community/support.html
...


In [4]:
#https://reactjs.org/blog/ 제외
urls = urls[0:3] + urls[4:]

In [5]:
def textParser(url,tags):
    bsObj = getHtmlToSoup(url)
    for tag in tags:
        nameList = bsObj.find_all(tag)
        nameList = [i.text for i in nameList]
        tagInterText[tag] += nameList
    return tagInterText

In [6]:
def dataToTable(data):
    tmp = []
    for key, texts in data.items():
        for text in texts:
            tmp.append([key,text])
    df = pd.DataFrame(data=tmp,columns=['tag','text'])
    return df

In [7]:
tagInterText = {'div':[],'p':[],'h1':[],'h2':[],'h3':[]}
dataToTable(textParser(urls[0],['div','p','h1','h2','h3']))

,tag,text
0,div,We want to hear from you!Take our 2021 Communi...
1,div,We want to hear from you!Take our 2021 Communi...
2,div,We want to hear from you!Take our 2021 Communi...
3,div,We want to hear from you!Take our 2021 Communi...
4,div,We want to hear from you!Take our 2021 Communi...
...,...,...
80,h3,"Learn Once, Write Anywhere"
81,h3,A Simple Component
82,h3,A Stateful Component
83,h3,An Application


In [8]:
#div는 사실 학습 내용과는 상관없고 반복적으로 같은 내용을 포함하고 있어, div제외한 p, h1, h2 tag의 내용을 모으기로 함.

In [9]:
tagInterText = {'p':[],'h1':[],'h2':[],'h3':[]}
tags = ['p','h1','h2','h3']
for url in tqdm(urls):
    df = dataToTable(textParser(url,tags))
    time.sleep(1)

100%|██████████| 121/121 [02:37<00:00,  1.30s/it]


In [10]:
df

,tag,text
0,p,A JavaScript library for building user interfaces
1,p,React makes it painless to create interactive ...
2,p,Declarative views make your code more predicta...
3,p,Build encapsulated components that manage thei...
4,p,Since component logic is written in JavaScript...
...,...,...
10072,h3,Going Big with React
10073,h3,Rethinking Best Practices
10074,h3,High Performance Functional DOM Programming
10075,h3,Developing User Interfaces With React


In [11]:
len(df)

10077

In [12]:
df.to_pickle("../data/reactInnerText.pkl")